In [12]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [10]:
ratings = pd.read_csv('tourism_rating.csv')
places = pd.read_csv('tourism_with_id.csv')
users = pd.read_csv('user.csv')
packages = pd.read_csv('package_tourism.csv')

In [13]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   User_Id        10000 non-null  int64
 1   Place_Id       10000 non-null  int64
 2   Place_Ratings  10000 non-null  int64
dtypes: int64(3)
memory usage: 234.5 KB


In [11]:
# Pivot table (User-Item matrix)
user_item_matrix = ratings.pivot_table(index='User_Id', columns='Place_Id', values='Place_Ratings')

In [16]:
# Fill NaN with 0
user_item_filled = user_item_matrix.fillna(0)

# User-User similarity
user_similarity = cosine_similarity(user_item_filled)
user_sim_df = pd.DataFrame(user_similarity, index=user_item_filled.index, columns=user_item_filled.index)

In [17]:
def get_similar_users(user_id, user_sim_df, top_n=5):
    # Get similarity scores for the user with all others
    sim_scores = user_sim_df.loc[user_id]
    # Remove self similarity
    sim_scores = sim_scores.drop(user_id)
    # Sort and get top N similar users
    top_users = sim_scores.sort_values(ascending=False).head(top_n)
    return top_users

def recommend_places(user_id, user_item_filled, user_sim_df, top_n=5):
    # Get top similar users
    similar_users = get_similar_users(user_id, user_sim_df)
    
    # Weighted sum of ratings from similar users
    weighted_ratings = np.zeros(user_item_filled.shape[1])
    sim_sum = 0
    for other_user, similarity in similar_users.items():
        weighted_ratings += similarity * user_item_filled.loc[other_user].values
        sim_sum += similarity

    if sim_sum == 0:
        print("No similar users found.")
        return []

    # Average weighted ratings
    weighted_avg_ratings = weighted_ratings / sim_sum

    # Places the user has already rated
    user_rated = user_item_filled.loc[user_id]
    already_rated = user_rated[user_rated > 0].index.tolist()

    # Recommend places not rated by the user
    recommendations = []
    for idx, place_id in enumerate(user_item_filled.columns):
        if place_id not in already_rated:
            recommendations.append((place_id, weighted_avg_ratings[idx]))

    # Sort recommendations by predicted rating
    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)

    return recommendations[:top_n]

# Example usage
user_id = user_item_filled.index[0]  # replace with any valid user ID
top_recommendations = recommend_places(user_id, user_item_filled, user_sim_df, top_n=5)

print(f"Top recommendations for User {user_id}:")
for place_id, score in top_recommendations:
    place_name = places.loc[places['Place_Id'] == place_id, 'Place_Name'].values[0]
    print(f"{place_name} (Place_Id: {place_id}) - Score: {score:.2f}")


Top recommendations for User 1:
Museum Kebangkitan Nasional (Place_Id: 63) - Score: 1.86
Kebun Teh Nglinggo (Place_Id: 166) - Score: 1.71
Dusun Bambu (Place_Id: 220) - Score: 1.53
Taman Miniatur Kereta Api (Place_Id: 327) - Score: 1.50
Pantai Sepanjang (Place_Id: 184) - Score: 1.47


In [29]:
places[places['City']=='Semarang']

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Unnamed: 11,Unnamed: 12
334,335,Candi Gedong Songo,Candi Gedong Songo (bahasa Jawa: ꦕꦤ꧀ꦝꦶ​ꦒꦼꦝꦺꦴꦁ​...,Budaya,Semarang,10000,4.5,NaN,"{'lat': -7.209886700000002, 'lng': 110.3421119}",-7.209887,110.342112,NaN,335
335,336,Grand Maerakaca,Masyarakat Jawa Tengah mungkin sudah tidak asi...,Taman Hiburan,Semarang,15000,4.4,NaN,"{'lat': -6.9605225, 'lng': 110.3863941}",-6.960522,110.386394,NaN,336
336,337,Kampung Pelangi,Kampung pelangi atau dalam bahasa Inggris dise...,Taman Hiburan,Semarang,3000,4.3,30.0,"{'lat': -6.988881200000001, 'lng': 110.4083781}",-6.988881,110.408378,NaN,337
337,338,Lawang Sewu,"Lawang Sewu (""Seribu Pintu"") (bahasa Jawa: ꦭꦮꦁ...",Budaya,Semarang,10000,4.6,NaN,"{'lat': -6.9839099, 'lng': 110.4104342}",-6.983910,110.410434,NaN,338
338,339,Sam Poo Kong Temple,"Sam Poo Kong (Hanzi: ; Pinyin: Sānbǎo Dòng), j...",Budaya,Semarang,35000,4.5,NaN,"{'lat': -6.996236599999999, 'lng': 110.398122}",-6.996237,110.398122,NaN,339
339,340,Desa Wisata Lembah Kalipancur,Wisata alam tengah menjadi sorotan bagi dunia ...,Taman Hiburan,Semarang,0,3.9,90.0,"{'lat': -7.0205245, 'lng': 110.3754605}",-7.020524,110.375461,NaN,340
340,341,Hutan Wisata Tinjomoyo Semarang,Awalnya taman wisata hutan Tinjomoyo Semarang ...,Cagar Alam,Semarang,3000,4.3,NaN,"{'lat': -7.0296837, 'lng': 110.3999611}",-7.029684,110.399961,NaN,341
341,342,Taman Kasmaran,Taman Kasmaran terletak di sebelah kiri Pasar ...,Taman Hiburan,Semarang,3000,4.5,90.0,"{'lat': -6.9909404, 'lng': 110.4065932}",-6.990940,110.406593,NaN,342
342,343,Pantai Baruna,kunjungi Pantai Baruna yang berada di Kota Sem...,Bahari,Semarang,3000,4.0,NaN,"{'lat': -6.945105199999999, 'lng': 110.3982398}",-6.945105,110.398240,NaN,343
343,344,Pantai Marina,"Pantai Marina (bahasa Jawa: ꦥꦱꦶꦱꦶꦂ​ꦩꦫꦶꦤ, trans...",Bahari,Semarang,3000,4.1,NaN,"{'lat': -6.948877, 'lng': 110.3893285}",-6.948877,110.389329,NaN,344


In [28]:
places.iloc[184-1]

Place_Id                                                      184
Place_Name                                       Pantai Sepanjang
Description     Siapa sangka, ternyata Yogyakarta pun punya “K...
Category                                                   Bahari
City                                                   Yogyakarta
Price                                                       10000
Rating                                                        4.4
Time_Minutes                                                120.0
Coordinate        {'lat': -8.136665599999999, 'lng': 110.5659962}
Lat                                                     -8.136666
Long                                                   110.565996
Unnamed: 11                                                   NaN
Unnamed: 12                                                   184
Name: 183, dtype: object